In [ ]:
import os
import re

# 目标文件夹路径
folder_path = r'\\apac.wdpr.disney.com\Corp\CNKC\Finance\Controllership\FINANCIAL REPORTING\FY_ 2025\Audits\Deloitte\CPL\CIP and FAIA\LTP\In sys report'

# 存储需要处理的文件列表
files_to_process = []

# 遍历文件夹,查找包含 'forecast' 且以 '.txt' 结尾的文件
for filename in os.listdir(folder_path):
    if 'forecast' in filename.lower() and filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        files_to_process.append(file_path)

# 定义提取 CY 数字的函数
def extract_cy_number(filename):
    """从文件名中提取 CY 后面的数字用于排序"""
    match = re.search(r'CY(\d+)', filename, re.IGNORECASE)
    if match:
        return int(match.group(1))
    return 0

# 按文件名中的 CY 数字排序
files_to_process.sort(key=lambda file_path: extract_cy_number(os.path.basename(file_path)))

# 支持的文件编码列表
encodings = ('utf-8', 'gbk', 'gb2312', 'latin-1')

# 处理每个文件
for file_path in files_to_process:
    filename = os.path.basename(file_path)
    print(f'正在处理: {filename}')
    
    content = None
    used_encoding = None
    
    # 尝试不同的编码读取文件
    for enc in encodings:
        try:
            with open(file_path, 'r', encoding=enc) as file:
                content = file.read()
                used_encoding = enc
                break
        except UnicodeDecodeError:
            continue
        except Exception:
            continue
    
    # 如果无法识别编码
    if content is None:
        print(f'  ✗ {filename} - 无法识别文件编码')
        continue
    
    try:
        # 将文件中的双引号 " 替换为空字符串(删除)
        content = content.replace('"', '')
        
        # 写回文件
        with open(file_path, 'w', encoding=used_encoding) as file:
            file.write(content)
        
        print(f'  ✓ {filename} - 引号已替换')
        
    except Exception as e:
        print(f'  ✗ {filename} - 处理失败: {e}')

print('\n处理完成!')
